In [72]:
from PIL import Image
import matplotlib.pyplot as plt
import numpy as np
import math
import numpy
import h5py
import PIL


In [154]:
h5_file_path = '/media/adalberto/Disk2/SSL-Pathology-Explorer/data_example/hdf5_epoch_40_projected_images.h5'
content = h5py.File(h5_file_path)
embedding = content['z_representation']
images    = content['images']


In [143]:
compression_rate = 4
image_height = int(images.shape[1]/compression_rate)
image_width  = int(images.shape[2]/compression_rate)
ss_width = image_width*20
columns = math.floor(ss_width/image_width)

In [144]:
all_rows = []
for i in range(len(images)):
    if i % columns == 0:
        all_rows.append([])
    current = all_rows[len(all_rows) - 1]
    current.append(images[i][::compression_rate, ::compression_rate, :])

In [145]:
ranges = []
for i in range(math.ceil(len(all_rows) / columns)):
    start = i * columns
    end = (i + 1) * columns
    if i == math.ceil(len(all_rows) / columns) - 1:
        end = len(all_rows)
    ranged = range(start, end)
    ranges.append(ranged)

In [146]:
ranges

[range(0, 20),
 range(20, 40),
 range(40, 60),
 range(60, 80),
 range(80, 100),
 range(100, 120),
 range(120, 140),
 range(140, 160),
 range(160, 180),
 range(180, 200),
 range(200, 220),
 range(220, 240),
 range(240, 260),
 range(260, 280),
 range(280, 300),
 range(300, 320),
 range(320, 340),
 range(340, 360),
 range(360, 380),
 range(380, 400),
 range(400, 420),
 range(420, 440),
 range(440, 460),
 range(460, 480),
 range(480, 500)]

In [147]:
len(ranges)

25

In [148]:
leftover = ss_width % columns
row_padding = []
for i in range(leftover):
    row_padding.append(0)
row_padding

[]

In [149]:
chunk_padding = []
for i in range(ss_width - columns * image_height):
    chunk_padding.append([])
    current = chunk_padding[len(chunk_padding) - 1]
    for p in range(ss_width):
        current.append(0)
len(chunk_padding)

0

In [150]:
len(all_rows[0])

20

In [151]:
ss_width - len(ranges[0]) * image_width

0

In [152]:
for range_counter in range(len(ranges)):
    full_combined = []
    for r in ranges[range_counter]:
        row_digits = all_rows[r]
        reshaped = []
        for i in range(len(row_digits)):
            digit = row_digits[i]
#             digit = digit.reshape((28,28))
            reshaped.append(digit)
        combined = []
        for i in range(image_height):
            combined.append([])
            current = combined[len(combined) - 1]
            for j in range(len(row_digits)):
                current.extend(reshaped[j][i])
            padding_needed = ss_width - len(row_digits) * image_width
            for p in range(padding_needed):
                current.append(0)
        full_combined.extend(combined)
    row_padding_needed = ss_width - len(full_combined)
    wtf = row_padding_needed
    for rp in range(row_padding_needed):
        current = []
        for p in range(ss_width):
            current.append(0)
        full_combined.append(current)
    test = full_combined
    rgba_combined = []
    for row in full_combined:
        rgba_combined.append([])
        current = rgba_combined[len(rgba_combined) - 1]
        for item in row:
            current.append((item[0], item[1], item[2]))
    rgba_combined = numpy.asarray(rgba_combined)
    rgba_combined = rgba_combined.astype(numpy.uint8)
    im = Image.fromarray(numpy.asarray(rgba_combined), mode='RGB')
    name = 'mnist_tile_solid_' + str(range_counter) + '.png'
    im.save(name)


In [156]:
import json
with open('mnist-embeddings.json', 'w') as outfile:
    json.dump(embedding[:].tolist(), outfile)

In [157]:
labels = [0]*embedding.shape[0]
with open('mnist_labels.json', 'w') as outfile:
    json.dump(labels, outfile)